# Accelerating deep neural networks with tensor decompositions

In this notebook we will show how to use TensorLy for accelerating convolutional layers in PyTorch.

This notebook is based on this blog post: https://jacobgil.github.io/deeplearning/tensor-decompositions-deep-learning

We will use the Tucker and CP decomositions to make low rank approximations of convolutional layers, and make them:
1. Faster.
2. Smaller.

Like pruning, these are tools that can be used to make the networks smaller and faster, and then try to restore accuracy by training further.

---

First lets get VGG19 from torch vision: 

In [38]:
from torchvision import models
model = models.vgg19(pretrained=True)
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)
    (18): MaxPool

The model has two parts. 

model.features is the convolutional part, containing only convolutional layers, ReLU and max pooling layers.


model.classifier contains several fully connected and ReLU layers and does the final classification on features extracted from model.features. 

We will be focusing on model.features. Lets look at one of the convolutional layers:

In [36]:
layer = model.features._modules['2']
print(layer)

Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


This layer performs 64 different convolutions (the output channels), 

where each convolution has a 3x3 spatial size and is done on an image with 64 channels (the input channels).

If we call our convolutional kernel K, the input image X and the layer bias b, then the output from the layer, V will be:

$$ V(x, y, t) = \sum_i \sum_j \sum_sK(i, j, s, t)X(x-i, x-j, s) + b(t) $$ 


The weights of the layer are stored in a 4 dimensional tensor:


In [25]:
print(layer.weight.size())

torch.Size([64, 64, 3, 3])


We will use TensorLy for applying tensor decompositions on this 4 dimensional weight tensor.


## CP Decomposition on convolutional layers

The idea here is from [1412.6553 Speeding-up Convolutional Neural Networks Using Fine-tuned CP-Decomposition](https://arxiv.org/abs/1412.6553)

We can approximate our 4 dimensional convolutional kernel with a CP decomposition of rank R:

$ K(i, j, s, t) = \sum_{r=1}^R K^x_r(i)K^y_r(j)K^s_r(s)K^t_r(t) $.

Plugging this into the formula for the convolutional layer output form above:

$ V(x, y, t) = \sum_r\sum_i \sum_j \sum_sK^x_r(i)K^y_r(i)K^s_r(s)K^t_r(t)X(x-i, y-j, s) $
$ = \sum_rK^t_r(t) \sum_i \sum_j K^x_r(i)K^y_r(j)\sum_sK^s_r(s)X(x-i, y-j, s) $ 

This gives us a recipe to do the convlution:

 1. First do a point wise (1x1xS) convolution with the kernek $K(r)$.
 
 This reduces the number of input channels from S to R.
 The convolutions will next be done on a smaller number of channels, making them faster.

 2. Perform seperable convolutions in the spatial dimensions with $K^x_r,K^y_r$. We can do this in PyTorch using grouped convolutions.

    **Like in [mobilenets](https://arxiv.org/abs/1704.04861) the convolutions are depthwise seperable, done in each channel separately.**
    **Unlike mobilenets the convolutions are also separable in the spatial dimensions.**

 3. Do another pointwise convolution to change the number of channels from R to T
 4. Finally, add the bias.
 
 
Notice the combination of pointwise and depthwise convolutions like in mobilenets. While with mobilenets you have to train a network from scratch to get this structure, here we can decompose an existing layer into this form.

As with mobile nets, to get the most speedup you will need a platform that has an efficient implementation of depthwise separable convolutions.

Now we can write a function that receives a PyTorch Conv2D layer, and creates the CP decompisition:

In [4]:
from tensorly.decomposition import parafac, partial_tucker
import numpy as np
import torch
import torch.nn as nn

def cp_decomposition_conv_layer(layer, rank):
    """ Gets a conv layer and a target rank, 
        returns a nn.Sequential object with the decomposition """

    # Perform CP decomposition on the layer weight tensorly. 
    X = layer.weight.data.numpy()
    last, first, vertical, horizontal = parafac(X, rank=rank, init = 'svd')

    pointwise_s_to_r_layer = torch.nn.Conv2d(in_channels = first.shape[0], \
            out_channels = first.shape[1],
            kernel_size = 1, \
            stride = 1,
            padding = 0,
            dilation = layer.dilation,
            bias = False)

    depthwise_vertical_layer = torch.nn.Conv2d(in_channels = vertical.shape[1], \
            out_channels = vertical.shape[1],
            kernel_size = (vertical.shape[0], 1),
            stride = 1,
            padding = (layer.padding[0], 0),
            dilation = layer.dilation,
            groups = vertical.shape[1],
            bias = False)

    depthwise_horizontal_layer = torch.nn.Conv2d(in_channels = horizontal.shape[1], \
            out_channels = horizontal.shape[1],
            kernel_size = (1, horizontal.shape[0]),
            stride = layer.stride,
            padding = (0, layer.padding[0]),
            dilation = layer.dilation,
            groups = horizontal.shape[1],
            bias = False)

    pointwise_r_to_t_layer = torch.nn.Conv2d(in_channels = last.shape[1], \
            out_channels = last.shape[0],
            kernel_size = 1, \
            stride = 1,
            padding = 0,
            dilation = layer.dilation,
            bias = True)
    pointwise_r_to_t_layer.bias.data = layer.bias.data

    # Transpose dimensions back to what PyTorch expects
    depthwise_vertical_layer_weights = np.expand_dims(np.expand_dims(\
        vertical.transpose(1, 0), axis = 1), axis = -1)
    depthwise_horizontal_layer_weights = np.expand_dims(np.expand_dims(\
        horizontal.transpose(1, 0), axis = 1), axis = 1)
    pointwise_s_to_r_layer_weights = np.expand_dims(\
        np.expand_dims(first.transpose(1, 0), axis = -1), axis = -1)
    pointwise_r_to_t_layer_weights = np.expand_dims(np.expand_dims(\
        last, axis = -1), axis = -1)

    # Fill in the weights of the new layers
    depthwise_horizontal_layer.weight.data = \
        torch.from_numpy(np.float32(depthwise_horizontal_layer_weights))
    depthwise_vertical_layer.weight.data = \
        torch.from_numpy(np.float32(depthwise_vertical_layer_weights))
    pointwise_s_to_r_layer.weight.data = \
        torch.from_numpy(np.float32(pointwise_s_to_r_layer_weights))
    pointwise_r_to_t_layer.weight.data = \
        torch.from_numpy(np.float32(pointwise_r_to_t_layer_weights))

    new_layers = [pointwise_s_to_r_layer, depthwise_vertical_layer, \
                    depthwise_horizontal_layer, pointwise_r_to_t_layer]
    return nn.Sequential(*new_layers)

print('Before the decomposition', layer)
layer_cp_decomposed = cp_decomposition_conv_layer(layer, rank=16)
print('After the decomposition', layer_cp_decomposed)

('Before the decomposition', Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
('After the decomposition', Sequential(
  (0): Conv2d (3, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): Conv2d (16, 16, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0), groups=16, bias=False)
  (2): Conv2d (16, 16, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), groups=16, bias=False)
  (3): Conv2d (16, 64, kernel_size=(1, 1), stride=(1, 1))
))


Using numpy backend.


# Tucker decomposition on convolutional layers

The idea here is from [*1511.06530 Compression of Deep Convolutional Neural Networks for Fast and Low Power Mobile Applications*](https://arxiv.org/abs/1511.06530).

Getting back to our kernel tensor K, we can approximate it using the Tucker decomposition:
$ K(i, j, s, t) = \sum_{r_1=1}^{R_1}\sum_{r_2=1}^{R_2}\sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{r_1 r_2 r_3 r_4} K^x_{r1}(i)K^y_{r2}(j)K^s_{r3}(s)K^t_{r4}(t) $

The Tucker decomposition has the useful property that it doesn't have to be decomposed along all the axis (modes).

Since the spatial dimensions are already quite small (3x3), it doesn't make a lot of sense to decompose those dimensions.

We can perform the decomposition along the input and output channels instead (a mode-2 decomposition):

$K(i, j, s, t) = \sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{i j r_3 r_4}(j)K^s_{r3}(s)K^t_{r4}(t)$

Like for CP decomposition, lets write the convolution formula and plug in the kernel decomposition: 
$ V(x, y, t) = \sum_i \sum_j \sum_s\sum_{r_3=1}^{R_3}\sum_{r_4=1}^{R_4}\sigma_{(i)(j) r_3 r_4}K^s_{r3}(s)K^t_{r4}(t)X(x-i, y-j, s) = \sum_i \sum_j \sum_{r_4=1}^{R_4}\sum_{r_3=1}^{R_3}K^t_{r4}(t)\sigma_{(i)(j) r_3 r_4} \sum_s\ K^s_{r3}(s)X(x-i, y-j, s) $ 

This gives us the following recipe for doing the convolution with Tucker Decomposition:

 1. Point wise convolution with $K^s_{r3}(s)$ for reducing the number of channels from S to $R_3$.

 2. Regular (not separable) convolution with $\sigma_{(i)(j) r_3 r_4} $.
 Instead of S input channels and T output channels like the original layer had,
 this convolution has $R_3$ input channels and $R_4$ output channels. If these ranks are smaller than S and T, this is were the speed gain comes from.

 3. Pointwise convolution with $K^t_{r4}(t)$ to get back to T output channels like the original convolution.
 4. Add the bias.

Now we can write a function that receives a PyTorch Conv2D layer, and creates the tucker decompisition:

In [7]:
def tucker_decomposition_conv_layer(layer, ranks):
    """ Gets a conv layer, 
        returns a nn.Sequential object with the Tucker decomposition.
    """
    
    core, [last, first] = \
        partial_tucker(layer.weight.data.numpy(), \
            modes=[0, 1], ranks=ranks, init='svd')

    # A pointwise convolution that reduces the channels from S to R3
    first_layer = torch.nn.Conv2d(in_channels = first.shape[0], \
            out_channels = first.shape[1],
            kernel_size = 1, \
            stride = 1,
            padding = 0,
            dilation = layer.dilation,
            bias = False)

    # A regular 2D convolution layer with R3 input channels 
    # and R3 output channels
    core_layer = torch.nn.Conv2d(in_channels = core.shape[1], \
            out_channels = core.shape[0],
            kernel_size = layer.kernel_size,
            stride = layer.stride,
            padding = layer.padding,
            dilation = layer.dilation,
            bias = False)

    # A pointwise convolution that increases the channels from R4 to T
    last_layer = torch.nn.Conv2d(in_channels = last.shape[1], \
            out_channels = last.shape[0],
            kernel_size = 1, \
            stride = 1,
            padding = 0,
            dilation = layer.dilation,
            bias = True)

    last_layer.bias.data = layer.bias.data


    # Transpose add dimensions to fit into the PyTorch tensors
    first = first.transpose((1, 0))
    first_layer.weight.data = torch.from_numpy(np.float32(\
        np.expand_dims(np.expand_dims(first.copy(), axis=-1), axis=-1)))
    last_layer.weight.data = torch.from_numpy(np.float32(\
        np.expand_dims(np.expand_dims(last.copy(), axis=-1), axis=-1)))
    core_layer.weight.data = torch.from_numpy(np.float32(core.copy()))

    new_layers = [first_layer, core_layer, last_layer]
    return nn.Sequential(*new_layers)

print('Before the decomposition', layer)
layer_tucker_decomposed = tucker_decomposition_conv_layer(layer, ranks=[16, 16])
print('After the decomposition', layer_tucker_decomposed)

('Before the decomposition', Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)))
('After the decomposition', Sequential(
  (0): Conv2d (3, 3, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): Conv2d (3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (2): Conv2d (16, 64, kernel_size=(1, 1), stride=(1, 1))
))


## Being careful with strides and dilations

Sometimes the convolutional layer will have a stride.
In this case, the last convolution among the decomposed parts, should have the original layer's stride, and the rest should have just a stride of one.
Since the last part of these decompositions is a 1x1 convolution, we can set the stride of the layer before it instead, to make it a bit faster. 

In case of dilations, all the spatial layers should preserve the dilation rate of the original layer.


## Accelerating the entire network
To accelerate the network, we can loop over the layers, and replace convolutional layers with their decomposition.
We will count the number of parameters in the network before and after the decomposition.

In [39]:
from functools import reduce # Valid in Python 2.6+, required in Python 3
import operator

def count_params(model):    
    return  sum(reduce(operator.mul, p.size(), 1) for p in model.parameters())

params_before = count_params(model.features)

for index, module in enumerate(model.features._modules):
    if index > 0:
        layer = model.features._modules[module]
        if type(layer) is torch.nn.Conv2d:
            ranks = [layer.weight.size(0)/2, layer.weight.size(1)/2]
            model.features._modules[module] = tucker_decomposition_conv_layer(layer, ranks)
        
print model.features

params_after = count_params(model.features)

print('Number of parameters before the decomposition', params_before)
print('Number of parameters after the decomposition', params_after)
print('Ratio', float(params_after) / params_before)

Sequential(
  (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Sequential(
    (0): Conv2d (64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d (32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d (32, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (5): Sequential(
    (0): Conv2d (64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d (32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (6): ReLU(inplace)
  (7): Sequential(
    (0): Conv2d (128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (2): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=(